In [53]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# import all the required data
train_df = pd.read_csv("./data/train.csv")
train_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_4116\3118064815.py:2: DtypeWarning: Columns (4,13,14,16,47) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("./data/train.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,points_y,position,positionText_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,20025,833,642,51,2,1,1,1,1,9.0,...,3.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,922731975
1,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923172525
2,20025,833,642,51,2,1,1,1,1,9.0,...,6.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,923833350
3,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,3,3,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,925195050
4,20025,833,642,51,2,1,1,1,1,9.0,...,10.33,4,4,0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...,Finished,926576775


In [3]:
test_df = pd.read_csv("./data/test.csv")
test_df.head()

C:\Users\krish\AppData\Local\Temp\ipykernel_4116\2707460454.py:1: DtypeWarning: Columns (13,16) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv("./data/test.csv")


,resultId,racerId,driverId,constructorId,number,grid,position_x,positionText_x,positionOrder,points,...,driverStandingsId,raceId_y,points_y,wins,constructorRef,company,nationality_y,url,status,result_driver_standing
0,23949,997,4,1,14,13,8,8,8,4.0,...,12474,93,16.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,298739826
1,23949,997,4,1,14,13,8,8,8,4.0,...,12494,94,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299218806
2,23949,997,4,1,14,13,8,8,8,4.0,...,12514,95,21.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,299697786
3,23949,997,4,1,14,13,8,8,8,4.0,...,12534,96,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300176766
4,23949,997,4,1,14,13,8,8,8,4.0,...,12554,97,25.0,0,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,+1 Lap,300655746


In [45]:
# dropping columns that are useless
new_train_df = train_df[
    ["result_driver_standing", "points_y", "company", "round",
     "status", "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId", "position"]]
new_train_df.head()


,result_driver_standing,points_y,company,round,status,wins,grand_prix,year,points,positionOrder,resultId,racerId,position
0,922731975,3.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,3
1,923172525,6.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,3
2,923833350,6.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,4
3,925195050,10.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,3
4,926576775,10.33,Alfa Romeo,1,Finished,0,British Grand Prix,1950,9.0,1,20025,833,4


In [46]:
new_test_df = test_df[
    ["result_driver_standing", "points_y", "company", "round",
     "status", "wins", "grand_prix", "year", "points", "positionOrder",
     "resultId", "racerId"]]
new_test_df.head()

,result_driver_standing,points_y,company,round,status,wins,grand_prix,year,points,positionOrder,resultId,racerId
0,298739826,16.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
1,299218806,21.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
2,299697786,21.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
3,300176766,25.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997
4,300655746,25.0,McLaren,9,+1 Lap,0,Austrian Grand Prix,2018,4.0,8,23949,997


In [47]:
finished_train = new_train_df[new_train_df["status"] == "Finished"]
finished_train = finished_train.drop(["status"], axis=1)
finished_train

,result_driver_standing,points_y,company,round,wins,grand_prix,year,points,positionOrder,resultId,racerId,position
0,922731975,3.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,3
1,923172525,6.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,3
2,923833350,6.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,4
3,925195050,10.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,3
4,926576775,10.33,Alfa Romeo,1,0,British Grand Prix,1950,9.0,1,20025,833,4
...,...,...,...,...,...,...,...,...,...,...,...,...
2830096,401167805,61.50,Red Bull,19,2,Brazilian Grand Prix,2013,18.0,2,22109,899,4
2830097,660484266,69.50,Red Bull,19,2,Brazilian Grand Prix,2013,18.0,2,22109,899,4
2830098,1409316096,4.00,Red Bull,19,0,Brazilian Grand Prix,2013,18.0,2,22109,899,8
2830099,1409846712,6.00,Red Bull,19,0,Brazilian Grand Prix,2013,18.0,2,22109,899,10


In [48]:

finished_test = new_test_df[new_test_df["status"] == "Finished"]

finished_test = finished_test.drop(["status"], axis=1)
finished_test

,result_driver_standing,points_y,company,round,wins,grand_prix,year,points,positionOrder,resultId,racerId
2053,2156580,0.0,Ferrari,10,0,British Grand Prix,2018,25.0,1,23962,998
2054,2683744,4.0,Ferrari,10,0,British Grand Prix,2018,25.0,1,23962,998
2055,3210908,5.0,Ferrari,10,0,British Grand Prix,2018,25.0,1,23962,998
2056,3738072,5.0,Ferrari,10,0,British Grand Prix,2018,25.0,1,23962,998
2057,4265236,5.0,Ferrari,10,0,British Grand Prix,2018,25.0,1,23962,998
...,...,...,...,...,...,...,...,...,...,...,...
352736,1880062640,6.0,Haas F1 Team,12,0,Belgian Grand Prix,2023,0.0,18,26083,1110
352737,1881105960,9.0,Haas F1 Team,12,0,Belgian Grand Prix,2023,0.0,18,26083,1110
352738,1881627620,9.0,Haas F1 Team,12,0,Belgian Grand Prix,2023,0.0,18,26083,1110
352739,1882149280,9.0,Haas F1 Team,12,0,Belgian Grand Prix,2023,0.0,18,26083,1110


In [49]:
finished_train = finished_train.fillna(0)
finished_test = finished_test.fillna(0)


In [50]:
# encoding all columns
label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_train[column] = le.fit_transform(finished_train[column])
    label_encoders[column] = le
    
label_encoders = {}
for column in ['company', 'grand_prix']:
    le = LabelEncoder()
    finished_test[column] = le.fit_transform(finished_test[column])
    label_encoders[column] = le


In [51]:
X = finished_train.drop(columns=['position'])
y = finished_train['position']

X_train = finished_train.drop(columns=['position'])
y_train = finished_train['position']

X_test = finished_train[X_train.columns]  
missing_cols = set(X_train.columns) - set(finished_test.columns)
extra_cols = set(finished_test.columns) - set(X_train.columns)
assert len(missing_cols) == 0, f"Test data is missing columns: {missing_cols}"
assert len(extra_cols) == 0, f"Test data has extra columns: {extra_cols}"

In [52]:
X = finished_train.drop(columns=['position'])
y = finished_train['position']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = {
    'objective': 'regression',
    'metric': 'rmse',  # RMSE for regression
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'device': 'gpu',  # Use GPU for training
    'gpu_platform_id': 0,  # Specify GPU platform ID (0 by default)
    'gpu_device_id': 0  # Specify GPU device ID (0 by default)
}

In [68]:
# train_data = lgb.Dataset(X_train, label=y_train)

# # Train the model
# num_round = 100
# bst = lgb.train(params, train_data, num_round)

# # Predict on test data
# predictions = bst.predict(X_test, num_iteration=bst.best_iteration)

# # Assuming 'test_data.csv' does not have 'position' column, you can append predictions
# finished_test['position'] = pd.Series(predictions)

# # Optionally, save predictions to a CSV file
# finished_test.to_csv('predictions.csv', index=False)


train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)

# Train the model
num_round = 3000
bst = lgb.train(params, train_data, num_round, valid_sets=[test_data])

# Predict on the test set
y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

# Evaluate the model
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 1279
[LightGBM] [Info] Number of data points in the train set: 766324, number of used features: 11
[LightGBM] [Info] Using requested OpenCL platform 0 device 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 10 dense feature groups (8.77 MB) transferred to GPU in 0.018558 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 9.154052
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 10 dense feature groups (7.02 MB) transferred to GPU in 0.011480 secs. 1 sparse feature groups
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 10 dense feature groups (7.02 MB) transferred to GPU in 0.011918 secs. 1 sparse feature groups
[LightGBM] [Info] Size of histogram bin 

c:\Users\krish\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [69]:
print(f'RMSE: {rmse}')
print(f'R²: {r2}')

# Optionally, save the model
bst.save_model('lightgbm_model.txt')

# Print sample predictions
predictions_df = X_test.copy()
predictions_df['actual_position'] = y_test
predictions_df['predicted_position'] = y_pred
print(predictions_df.head())

RMSE: 3.0246210176602326
R²: 0.8504801027395025
         result_driver_standing  points_y  company  round  wins  grand_prix  \
2166467                 4063150      20.0       82      5     0          34   
2820112              1526965597       6.0       81     16     0          18   
48378                 855494094       0.0       35      3     0          19   
2758364              1484778366      10.0       24      1     0           2   
2069060                 6663228      12.0       87     13     0          17   

         year  points  positionOrder  resultId  racerId  actual_position  \
2166467  2005     5.0              4      1222       75                5   
2820112  2013     1.0             10     22051      896               11   
48378    1956     0.0              8     18921      786               63   
2758364  2013    18.0              2     21713      880               15   
2069060  2003     5.0              4      2118      120                5   

         predicted_p